In [28]:
# import pyscf
from pyscf import gto, scf
from pyscf import lo
import numpy as np
import scipy as sp

In [134]:
class MoleculeFeatureExtractor:

    def __init__(self, mol):
        self.mol = mol

    @staticmethod
    def localize_orbitals_separately(mol, mo_coeff, mo_occ):

        """
        Localize occupied and virtual molecular orbitals separately using Boys method.

        Args:
            mol: pyscf.gto.Mole
                Molecule object.
            mf: pyscf.scf.hf.SCF
                Mean-field (SCF) calculation result.

        Returns:
            numpy.ndarray:
                stacked localized occupied and virtual orbitals.
        """

        #Create boolean masks
        occ_idx = mo_occ > 0
        vir_idx = mo_occ == 0

        # Use masks to select occupied and virtual orbitals
        occupied_orbitals_coeffs = mo_coeff[:, occ_idx]
        virtual_orbitals_coeffs = mo_coeff[:, vir_idx]

        #.pipek.PipekMezey, .edmiston.EdmistonRuedenberg

        localized_occupied_orbitals_method = lo.pipek.PipekMezey(mol, occupied_orbitals_coeffs)
        localized_virtual_orbitals_method = lo.pipek.PipekMezey(mol, virtual_orbitals_coeffs)        

        localized_occupied_orbitals_coeffs = localized_occupied_orbitals_method.kernel()
        localized_virtual_orbitals_coeffs = localized_virtual_orbitals_method.kernel()

        print(f"localized_occupied_orbitals_coeffs shape {localized_occupied_orbitals_coeffs.shape}")
        print(f"localized_virtual_orbitals_coeffs.shape {localized_virtual_orbitals_coeffs.shape}")
        print(f"np.linalg.pinv(occupied_orbitals_coeffs): {np.linalg.pinv(occupied_orbitals_coeffs).shape}")
        print(f"np.linalg.pinv(virtual_orbitals_coeffs){np.linalg.pinv(virtual_orbitals_coeffs).shape}")

        U_occ =  np.linalg.pinv(occupied_orbitals_coeffs) @ localized_occupied_orbitals_coeffs
        U_vir = np.linalg.pinv(virtual_orbitals_coeffs) @ localized_virtual_orbitals_coeffs
        U = sp.linalg.block_diag(U_occ, U_vir)

        loc_mo_coeffs = np.hstack([localized_occupied_orbitals_coeffs, localized_virtual_orbitals_coeffs])

        return loc_mo_coeffs, U

    @staticmethod
    def population_analysis(mol, C_loc, mf):

        """
        Perform Mulliken population analysis on molecular orbitals 
        to determine which atoms each orbital is centered on.
        Args:
            mol: pyscf.gto.Mole
                Molecule object.
            C_loc: numpy.ndarray
                localized molecular orbitals coefficients.
            mf: pyscf.scf.hf.SCF
                Mean-field (SCF) calculation result.

        Returns:
            Tuple([List[str], List[str], List[float]]):
                atoms_0: list of most-contributing atom symbols per orbital.
                atoms_1: list of second-most -contributing atom symbols per orb.
                distances: list of distances between those atom pairs.
        """

        # Get overlap matrix
        S = mf.get_ovlp()

        # Atom and AO info
        n_aos = mol.nao
        n_atoms = mol.natm

        atoms_0 = []
        atoms_1 = []
        distances = []
        
        for i in range(C_loc.shape[1]):
            c = C_loc[:, i] # i-th localized orbital
            pop_per_atom = np.zeros(n_atoms)

            for A in range(n_atoms):
                ao_slice = mol.aoslice_by_atom()[A]
                p0, p1 = ao_slice[2], ao_slice[3]

                for mu in range(p0, p1):
                    for nu in range(n_aos):
                        pop_per_atom[A] += c[mu] * c[nu] * S[mu, nu]

            indices = pop_per_atom.argsort()[-2:][::-1]
            idx0, idx1 = indices[0], indices[1]
            symb_0 = mol.atom_symbol(idx0)
            symb_1 = mol.atom_symbol(idx1)
            coord_0 = mol.atom_coord(idx0)  
            coord_1 = mol.atom_coord(idx1)
            
            atoms_0.append(symb_0)
            atoms_1.append(symb_1)
            distances.append(np.linalg.norm(coord_1 - coord_0))
            
        return atoms_0, atoms_1, distances

    @staticmethod
    def determine_orbital_type(mol, C_loc):

        """
        calculate the expectation values of Lz^2 for the molecular orbitals 
        to determine their character.

        Args:
            mol: pyscf.gto.Mole
                Molecule object.
            C_loc: numpy.ndarray
                localized molecular orbitals coefficients.
            
        Returns:
            list of str:
                labels: Labels of the type of MO (σ, π, δ, or mixed).
        """

        lz_3comp = gto.moleintor.getints('int1e_cg_irxp_sph', mol._atm, mol._bas, mol._env, comp = 3)
        lz_matrix = lz_3comp[2]
        lz_squared = lz_matrix.conj().T @ lz_matrix 
        lz_expect = np.diag(C_loc.conj().T @ lz_squared @ C_loc).real

        labels = []
        print(f"lz_expect: {lz_expect}")
        for lz_val in lz_expect:

            if abs(lz_val) < 0.1:
                label = 'σ'
            elif abs(lz_val - 1.0) < 0.1:
                label = 'π'
            elif abs(lz_val - 4.0) < 0.1:
                label = 'δ'
            else:
                label = 'mixed'

            labels.append(label)
        return labels

    @staticmethod
    def calculate_energy(mf, U):

        """
        Calculate the energies of the localized molecular orbitals.

        Args:
            mf: pyscf.scf.hf.SCF
                Mean-field (SCF) calculation result.
            U: np.ndarray
               Unitary matrix used to localize the MOs.
            
        Returns:
            List[float]:
                loc_mo_energies: Energies of the localized MOs.
        """

        mo_energies = mf.mo_energy
        loc_mo_energies = np.sum(np.abs(U)**2 * mo_energies[np.newaxis, :], axis=1)
        print(loc_mo_energies)
        return loc_mo_energies
        
    def extract_molecule_features(self):

        """
        Extract molecule features

        Args:

        Returns:
            Tuple([List[str], List[str], List[float], List[str], List[float]):
                atoms_0: list of most-contributing atom symbols per orbital.
                atoms_1: list of second-most -contributing atom symbols per orb.
                distances: list of distances between those atom pairs.
                labels: Labels of the type of MO (σ, π, δ, or mixed).
                mo_energies: Energies of the localized MOs.
        """
        mf = scf.RHF(self.mol)
        mf.kernel()
        mo_coeff = mf.mo_coeff
        mo_occ = mf.mo_occ
        C_loc, U = MoleculeFeatureExtractor.localize_orbitals_separately(self.mol, mo_coeff, mo_occ)
        atoms_0, atoms_1, distances = MoleculeFeatureExtractor.population_analysis(self.mol, C_loc, mf)
        print(f"atoms 0: {atoms_0}")
        print(f"atoms 1: {atoms_1}")
        print(f"distances: {distances}")
        labels = MoleculeFeatureExtractor.determine_orbital_type(self.mol, C_loc)
        print(labels)
        mo_energies = MoleculeFeatureExtractor.calculate_energy(mf, U)

In [137]:
from pyscf import gto, scf, cc

"""
mol = gto.Mole()
mol.atom = '''
H  0.0000  0.0000  -2.261
C  0.0000  0.0000  -1.203
C  0.0000  0.0000   1.203
H  0.0000  0.0000   2.261
'''
mol.basis = 'sto-3g'
mol.charge = 0
mol.spin = 0
mol.build()
"""

mol = gto.Mole()
mol.atom = '''
O1 0 0 0
O2 0 0 2.28
'''
mol.unit = 'B'
mol.basis = 'sto-3g'
mol.charge = 0
mol.spin = 0
mol.build()

In [138]:
c = MoleculeFeatureExtractor(mol)
c.extract_molecule_features()

converged SCF energy = -147.551024855249
localized_occupied_orbitals_coeffs shape (10, 8)
localized_virtual_orbitals_coeffs.shape (10, 2)
np.linalg.pinv(occupied_orbitals_coeffs): (8, 10)
np.linalg.pinv(virtual_orbitals_coeffs)(2, 10)
atoms 0: ['O2', 'O1', 'O2', 'O2', 'O2', 'O1', 'O1', 'O1', 'O1', 'O1']
atoms 1: ['O1', 'O2', 'O1', 'O1', 'O1', 'O2', 'O2', 'O2', 'O2', 'O2']
distances: [2.28, 2.28, 2.28, 2.28, 2.28, 2.28, 2.28, 2.28, 2.28, 2.28]
lz_expect: [8.54755264e-03 8.54755264e-03 5.76408444e-01 4.42889337e-02
 9.47163514e-01 4.42889367e-02 5.76408462e-01 9.47163511e-01
 8.47182246e-01 8.47182811e-07]
['σ', 'σ', 'mixed', 'σ', 'π', 'σ', 'mixed', 'π', 'mixed', 'σ']
[-20.25449472 -20.25258236  -0.91262583  -0.77477001  -0.67037308
  -0.90193605  -1.03788307  -0.67037665   0.18918485   0.73616753]
